# Sigmoid 문제점

- 0~1 범주를 가지는 activation function이 backpropagation이 되면서 Vanishing Gradient 현상이 발생

Optimizer마다의 성능을 그림으로 체크 가능한 gif URL
- http://www.denizyuret.com/2015/03/alec-radfords-animations-for.html

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
learning_rate=1e-3
training_epochs=25
batch_size=100

### dsets에서 불러올 때 transform을 비교해보기
- 결론 : 차이가 없다..? 굳이 적지 않아도 자동 텐서로 불러와짐

In [4]:
mnist_train=dsets.MNIST(root='MNIST_data/', train=True, download=True)
print(mnist_train.data[0].shape)
print(mnist_train.data[0].type)

torch.Size([28, 28])
<built-in method type of Tensor object at 0x000001AD591F35E8>


In [5]:
mnist_train=dsets.MNIST(root='MNIST_data/', train=True, transform=transforms.ToTensor(), download=True)
mnist_test=dsets.MNIST(root='MNIST_data/', train=False, transform=transforms.ToTensor(), download=True)

In [6]:
print(mnist_train.data[0].shape)
print(mnist_train.data[0].type)

torch.Size([28, 28])
<built-in method type of Tensor object at 0x000001AD59201688>


In [7]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)
linear = torch.nn.Linear(784,10,bias=True).to(device)
print(torch.nn.init.normal_(linear.weight))

Parameter containing:
tensor([[-1.2901,  0.3930, -0.1891,  ..., -0.1626, -0.3275,  0.5608],
        [-1.6757, -0.2454,  0.9221,  ...,  0.9559,  0.6160, -0.4226],
        [ 0.2444,  1.4810, -2.0906,  ...,  0.1206,  1.0044, -0.3737],
        ...,
        [ 0.6234,  1.8019, -2.7563,  ..., -0.5889, -0.5576,  0.7360],
        [-0.2871, -1.3313, -2.2248,  ...,  0.0309,  0.9180, -0.1482],
        [ 0.7678,  0.6624, -0.5362,  ...,  0.2338,  0.3688, -0.7182]],
       device='cuda:0', requires_grad=True)


In [8]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(linear.parameters(), lr=learning_rate)

In [9]:
# 60000/100 = 600
print(len(data_loader))

600


In [10]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost=0
    
    for X,Y in data_loader:
        X = X.view(-1,28*28).to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = linear(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch        
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

Epoch: 0001 cost = 5.656310081
Epoch: 0002 cost = 1.699869871
Epoch: 0003 cost = 1.121589899
Epoch: 0004 cost = 0.883372605
Epoch: 0005 cost = 0.750498414
Epoch: 0006 cost = 0.663257360
Epoch: 0007 cost = 0.601353467
Epoch: 0008 cost = 0.554276466
Epoch: 0009 cost = 0.518754542
Epoch: 0010 cost = 0.489626855
Epoch: 0011 cost = 0.465853274
Epoch: 0012 cost = 0.446376979
Epoch: 0013 cost = 0.429088771
Epoch: 0014 cost = 0.414358288
Epoch: 0015 cost = 0.401437402
Epoch: 0016 cost = 0.390579313
Epoch: 0017 cost = 0.379998147
Epoch: 0018 cost = 0.371015072
Epoch: 0019 cost = 0.362883240
Epoch: 0020 cost = 0.355653346
Epoch: 0021 cost = 0.348467052
Epoch: 0022 cost = 0.342722595
Epoch: 0023 cost = 0.336138189
Epoch: 0024 cost = 0.331430674
Epoch: 0025 cost = 0.326691180


In [11]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(-1,28*28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = linear(X_test)
    correct_prediction = torch.argmax(prediction,1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = linear(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy: 0.8887999653816223
Label:  8
Prediction:  3


C:\Users\la\Anaconda3\envs\tf20\lib\site-packages\torchvision\datasets\mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\la\Anaconda3\envs\tf20\lib\site-packages\torchvision\datasets\mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


# 다층 레이어 쌓기

In [12]:
linear1 = torch.nn.Linear(784,256,bias=True).to(device)
linear2 = torch.nn.Linear(256,128,bias=True).to(device)
linear3 = torch.nn.Linear(128,64,bias=True).to(device)
linear4 = torch.nn.Linear(64,10,bias=True).to(device)
relu = torch.nn.ReLU()

torch.nn.init.normal_(linear1.weight)
torch.nn.init.normal_(linear2.weight)
torch.nn.init.normal_(linear3.weight)
torch.nn.init.normal_(linear4.weight)

model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3, relu, linear4).to(device)

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [13]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost=0
    
    for X,Y in data_loader:
        X = X.view(-1,28*28).to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch        
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

Epoch: 0001 cost = 801.790222168
Epoch: 0002 cost = 143.039810181
Epoch: 0003 cost = 85.112846375
Epoch: 0004 cost = 57.566673279
Epoch: 0005 cost = 41.232223511
Epoch: 0006 cost = 30.384174347
Epoch: 0007 cost = 22.720020294
Epoch: 0008 cost = 17.292089462
Epoch: 0009 cost = 13.399991035
Epoch: 0010 cost = 10.140534401
Epoch: 0011 cost = 7.649079800
Epoch: 0012 cost = 6.193267822
Epoch: 0013 cost = 4.639629841
Epoch: 0014 cost = 3.744567394
Epoch: 0015 cost = 2.982141972
Epoch: 0016 cost = 2.320754766
Epoch: 0017 cost = 1.865953207
Epoch: 0018 cost = 1.647352576
Epoch: 0019 cost = 1.416785121
Epoch: 0020 cost = 1.410082221
Epoch: 0021 cost = 1.119329333
Epoch: 0022 cost = 0.909280002
Epoch: 0023 cost = 0.994037271
Epoch: 0024 cost = 0.820456922
Epoch: 0025 cost = 0.804532468
